In [2]:
import awkward as ak
print(ak.__version__)

0.12.0


In [3]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
print(myarray[1:])
# returns <JaggedArray [[] [4.4 5.5]] at 7f02018afc18>
print(myarray[100:])
# returns <JaggedArray [] at 7f020c214438>

[[] [4.4 5.5]]
[]


In [4]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
indexes = ak.JaggedArray.fromiter([[1, 2, 2, 0], [], [1]])
myarray[indexes]
# returns <JaggedArray [[3.3 3.3 1.1] [] [5.5]] at 7f02018afa58>

<JaggedArray [[2.2 3.3 3.3 1.1] [] [5.5]] at 0x7f1b71f976a0>

In [6]:
myarray = ak.JaggedArray.fromcounts([3, 0, 2], ak.Table(
              x=[1, 2, 3, 4, 5],
              y=[1.1, 2.2, 3.3, 4.4, 5.5],
              z=[True, False, True, False, False]))
print(myarray["x"])
# returns <JaggedArray [[1 2 3] [] [4 5]] at 7f020e8122b0>
print(myarray[["x", "y"]])
# returns <JaggedArray [[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]] at 7f02018af860>
print(myarray[["x", "y"]].columns)
# returns ['x', 'y']
print(myarray.count())

[[1 2 3] [] [4 5]]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
['x', 'y']
[<Row 0> <Row 1> <Row 2>]
